# Customer Churn – KPI Analysis

This notebook calculates key churn metrics and explores how churn varies
across customer segments.


In [1]:
import pandas as pd

df = pd.read_csv("../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

df = df.dropna(subset=["TotalCharges"])


In [2]:
df.shape

(7032, 21)

In [3]:
# Convert churn to binary
df["ChurnFlag"] = df["Churn"].map({"Yes": 1, "No": 0})

In [4]:
#Churn metric for how many customers leave on average
overall_churn_rate = df["ChurnFlag"].mean()
overall_churn_rate

0.26578498293515357

In [5]:
churn_by_contract = (
    df.groupby("Contract")["ChurnFlag"]
      .mean()
      .sort_values(ascending=False)
)

churn_by_contract

Contract
Month-to-month    0.427097
One year          0.112772
Two year          0.028487
Name: ChurnFlag, dtype: float64

### Insight:
Customers on month-to-month contracts churn at a significantly higher rate than those on one- or two-year contracts, suggesting contract length is a major driver of retention.

In [6]:
#Tenure buckets
df["TenureGroup"] = pd.cut(
    df["tenure"],
    bins = [0, 12, 24, 36, 48, 60, 72],
    labels=["0–12", "12–24", "24–36", "36–48", "48–60", "60+"]
)

churn_by_tenure = (
    df.groupby("TenureGroup", observed=False)["ChurnFlag"]
      .mean()
)
churn_by_tenure

TenureGroup
0–12     0.476782
12–24    0.287109
24–36    0.216346
36–48    0.190289
48–60    0.144231
60+      0.066098
Name: ChurnFlag, dtype: float64

### Insight: Churn by tenure

Customer churn is heavily front-loaded, with nearly half of customers leaving within the first 12 months. Churn decreases consistently with longer tenure, indicating that retention efforts should focus on early-life customers, particularly during the first year.



In [7]:
df.groupby(["Contract", "TenureGroup"], observed=False)["ChurnFlag"].mean().unstack()

TenureGroup,0–12,12–24,24–36,36–48,48–60,60+
Contract,,,,,,
Month-to-month,0.513541,0.377205,0.325103,0.335443,0.277778,0.222222
One year,0.105691,0.081218,0.080000,0.130597,0.137072,0.121406
Two year,0.000000,0.000000,0.020833,0.022472,0.039711,0.031440


### Insight

Churn is highest among month-to-month customers, particularly within the first 12 months, where over half of customers leave. Customers on one and two-year contracts exhibit significantly lower churn across all tenure groups. This suggests that contract length plays a critical role inearly customer retention, and that encouraging longer contracts early in the customer lifecycle could substantially reduce churn.